<H1 style=text-align:center;><b>Projet Maths-Info : Traveling Salesman Problem</b></H1>
<h1 style=text-align:center;>(AMU) EADS 2024-2025</h1>

<h2 style=text-align:center;><i>Projet présenté par : Alexandre ROMANO / Victoria BOUCHET / Vahé TILDIAN</i></h2>

<br><br><br>

## <u>Introduction</u>

...


<br>

## <u>Fonctions utiles</u>

...

### distance.py

...

In [ ]:
# Distance total d'un chemin
def total_path_distance(path, distance_matrix):
    path_length = 0
    for i in range(len(path) - 1):
        path_length += distance_matrix[path[i]][path[i + 1]]
    path_length += distance_matrix[path[-1]][path[0]]
    return path_length

### generate.py

...

In [ ]:
import numpy as np
from dataclasses import dataclass
import random


### Génération d'un ensemble de villes et la distance entre elles ###
## Peut être généré en faisant une matrice tel que pour la i-ème ville et j-ième ville on ait 
# [len(i,i) = 0, len(i,j), len(i,j+1) ... ]
# [len(j,i), len(j,j) = 0, len(j+1,j+1) ... ]

@dataclass
class Ville:
    
    def __init__(self, x, y, id):
        self.x = x
        self.y = y
        self.id = id

    def distance_to(self, other: 'Ville'):
        return np.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)

def distance_matrix(matrix_size): # => Nombre de villes
    villes = [Ville(random.randrange(0, 100), random.randrange(0, 100), i) for i in range(matrix_size)] #uniform -> randrange pour du int au lieu de float
    distance_matrix = np.zeros((len(villes), len(villes)))

    for i, ville1 in enumerate(villes):
        for j, ville2 in enumerate(villes):
            distance_matrix[i][j] = ville1.distance_to(ville2)

    #TEST : retour des coordonénes pour tester la visualisation
    coords = [(v.x, v.y) for v in villes]
    
    return distance_matrix, coords

### quality.py

...

In [ ]:
### Fonction qui testera la qualité et les performances d'un algorithme TSP ###
import functions.distance as distance
import time

def gap(total_path_distance, lower_bound):
    return f"{(total_path_distance - lower_bound) / lower_bound * 100} %"

def evaluate_quality(name, algo, distance_matrix, lower_bound):
    start_time = time.process_time()
    
    path = distance.total_path_distance(algo, distance_matrix)
    print(f'Route la plus courte pour l\'algorithme " {name} " : \n{algo}')
    print(f"Longueur totale du chemin trouvé : {path}")
    print(f"Qualité de la solution par rapport à la borne inférieure (Nearest Neighbor) : {gap(path, lower_bound)} de distance en moins")
    
    end_time = time.process_time()
    print(f"Temps de calcul : {end_time} s\n")

### visualisation.py

...

In [ ]:
import matplotlib.pyplot as plt

def plot_path(coord, paths, names):
    """
    Visualisation des chemins obtenus avec :
        coord = Liste des coordonnées
        path = Ordre des villes visités 
        name = nom de la technique utilisée
    
    """
    fig, axis = plt.subplots(1, len(paths), figsize=(5 * len(paths), 6))
    
    if len(paths) == 1:
        axis = [axis]

    for ax, path, name in zip(axis, paths, names):
        path_coords = [coord[i] for i in path] + [coord[path[0]]]
        
        # Séparation des coordonnées x et y pour créer deux listes avec chacunes d'entre elles
        x_path, y_path = zip(*path_coords)
        
        # --- Visualisation du chemin obtenu ---
        #plt.figure(figsize=(8,6))
        ax.plot(x_path, y_path, 'ro-', label ='Chemin TSP', markersize=5)  #b --> couleur bleue; o --> marqueur cercle ; "-" --> relié par des lignes 
        
        # --- Affichage des numéros de villes ---
        # On parcourt les coordonnées originales pour placer chaque numéros
        for i, (x,y) in enumerate(coord):
            #Affiche le numéro de la ville 'i' (on décale un peu pour la lisibilité)
            ax.text(x,y + 0.05, str(i), color='blue', fontsize=12, ha='center')

        # Mise en évidence du point de départ
        start_city_coords = coord[path[0]]
        ax.plot(start_city_coords[0], start_city_coords[1], 'go', markersize=10, label="Ville de départ")


        # --- Mise en forme du graphique ---
        ax.set_title(f"Méthode {name}")
        ax.set_xlabel("Coordonnée x")       #Axe des x
        ax.set_ylabel("Coordonnée y")       #Axe des y
        ax.legend()
        ax.grid(True)                   #Grille
        ax.axis('equal')                #Evite les distorsions
    
    plt.tight_layout()
    plt.show()                       #Affichage de la fenêtre

<br>

## <u>Implémentation des Heuristiques</u>

...

### Nearest Neighbor (neighbor.py)

...

In [ ]:
### Nearest Neighbor ###
def nearest_neighbor(distance_matrix):
    n = len(distance_matrix)
    unvisited = set(range(n))
    current = 0
    route = [current]
    unvisited.remove(current)

    while unvisited:
        nearest = min(unvisited, key=lambda city: distance_matrix[current][city])
        route.append(nearest)
        current = nearest
        unvisited.remove(nearest)

    route.append(route[0])
    return route

### Cheapest Insertion (insertion.py)

...


In [ ]:
#Code de l'heuristique d'insertion minimale
import functions.distance as distance

def cheapest_insertion(dist_matrix):
    """
    METHODE D'INSERTION : 
    Construit une solution approchée du TSP par heuristique d'insertion

    La fonction prend en entrée une matrice carrée des distances entre les villes et
    renvoie un ordre de visite minimisant approximativment la distance totale du chemin

    """
    n = len(dist_matrix)                                 #Nombre total des villes
    unvisited = list(range(n))                           #Liste des idices des villes non inserees dans le circuit
    path = [unvisited.pop(0), unvisited.pop(0)]          #Initialisation du chemin avec les deux premières villes


    while unvisited:
        #Recherhce la meilleure ville à inserer et de sa position optimale

        best_cost = float('inf')                            #Init avec la valeur infini pour technique borne inférieur
        best_place = None
        to_insert = None                                    #Ville à insérer lorsque celle-ci augmente le moins le coût total du chemin

        for v in unvisited:                                 #Insertion de chaque ville restantes une à une
            for i in range(len(path)):
                test_path = path[:i+1] + [v] + path[i+1:]   #Création d'un nouveau chemin hypothétique ave la ville v
                new_cost = distance.total_path_distance(test_path, dist_matrix)

                #Si l'insertion donne un meilleur cout alors on la garde en mémoire comme meilleure option
                if new_cost < best_cost:
                    best_cost = new_cost    
                    best_place = i+1
                    to_insert = v

        #Insertion de la meilleure ville à la meilleure position
        path.insert(best_place, to_insert)
        unvisited.remove(to_insert)

    path.append(path[0])
    return path


### 2-opt (twoopt.py)

...

In [ ]:
#Code permettant de résoudre le TSP avec la méthode 2-opt
#Se basant sur des 2-permutations

def two_opt(path, dist_matrix):
    """
    METHODE 2-OPT : Amélioration d'un chemin du TSP
    Prend en entrée un chemin (une liste d'indices) et une matrice de distance
    Renvoie un chemin amélioré à l'aide de 2-permutations (inversion d'arête)
    """
    n = len(path)
    improve = True              #Initialisation du booléen d'amélioration à Vrai

    max_iterations = 1000       #Maximum d'itération avant arrêt de l'algorithme
    iteration = 0               #Comptage d'itération

    while improve and iteration < max_iterations:         #Evite que le calcul ne boucle trop longtemps lorsque la solution initiale est mauvaise
        improve = False
        iteration += 1
    
        for i in range(n-1):
                for j in range(i+2, n if i>0 else n-1):   #Empêche les inversions invalides
                    a, b = path[i], path[i+1]             # a --> b
                    c, d = path[j], path[(j+1)%n]         # c --> d
                    
                    #Calcul du gain de distance
                    delta = -dist_matrix[a][b] -dist_matrix[c][d] + dist_matrix[a][c] + dist_matrix[b][d]

                    if delta < 0 : #Si delta est signifcativement négatif alors le chemin est plus court, on procède à l'échange
                        path[i+1:j+1] = reversed(path[i+1:j+1])
                        improve = True

        if iteration >= max_iterations :
             print(f"Avertissement : Arret apres {iteration} iterations, limite maximum\n")
    return path

### Simulated Annealing (annealing.py)

...

In [ ]:
import math
import random

def total_distance(route, coords): # à enlever ? Redondance avec quality
    dist = 0
    for i in range(len(route)):
        ville1 = coords[route[i]]
        ville2 = coords[route[(i + 1) % len(route)]]
        dist += math.sqrt((ville1[0] - ville2[0]) ** 2 + (ville1[1] - ville2[1]) ** 2)
    return dist

def simulated_annealing(coords, path, initial_temp, cooling_rate):   
    current_path = path
    current_distance = total_distance(current_path, coords)

    best_path = current_path[:]
    best_distance = current_distance

    T = initial_temp
    max_iterations = 50000

    for iteration in range(max_iterations):
        # 2-opt
        i = random.randint(0, len(coords) - 2)
        k = random.randint(i + 1, len(coords) - 1)
        
        new_path = current_path[:i] + current_path[i:k+1][::-1] + current_path[k+1:]
        new_distance = total_distance(new_path, coords)
        delta = new_distance - current_distance
        
        # Est-ce une meilleure solution ?
        if delta < 0 or random.random() < math.exp(-delta / T):
            current_path = new_path
            current_distance = new_distance
            # Mise à jour de la meilleure solution
            if new_distance < best_distance:
                best_path = new_path
                best_distance = new_distance

        # Réduit la température.
        T *= cooling_rate
        
        """ if iteration % 1000 == 0: # Pour suivre l'avancée
            print(f"Iteration {iteration}: distance actuelle = {current_distance:.4f}, meilleure distance = {best_distance:.4f}") """

    index = best_path.index(0)                          # L'index de la ville 0
    best_path = best_path[index:] + best_path[:index]   # On réarrange le chemin pour commencer par la ville 0 (le chemin reste le même)
    best_path.append(0)                                 # Retour à la ville de départ (0)
    return best_path, best_distance

if __name__ == '__main__': # TEST
    nb_villes = 10
    cities = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(nb_villes)]
    
    # Haute température de départ pour une exploration large des solutions, refroidissement lent pour une convergence graduelle
    initial_temp = 10000
    cooling_rate = 0.995
    
    path = [0, 2, 9, 3, 7, 6, 5, 1, 4, 8]
    
    best_path, best_distance = simulated_annealing(cities, path, initial_temp, cooling_rate)
    
    print("\nBest path :")
    print(best_path)
    print(f"Total distance: {best_distance:.4f}")

<br>

## <u>Main</u>

...

In [ ]:
"""
PROJET MATHS-INFO
"""

### IMPORTS ###
import functions.generate as generate
import algorithms.neighbor as neighbor
import algorithms.insertion as insertion
import algorithms.twoopt as twoopt
import algorithms.annealing as annealing
import functions.visualisation as vs
import functions.quality as qc
import functions.distance as distance

### MAIN ###
distance_matrix, coords = generate.distance_matrix(50)

print("Matrice des distances entre les villes: \n")
print(distance_matrix)
print("\n")

algo1 = neighbor.nearest_neighbor(distance_matrix)
lower_bound = distance.total_path_distance(algo1, distance_matrix) # Référence pour les autres algorithmes
qc.evaluate_quality("Nearest Neighbor", algo1, distance_matrix, lower_bound)

algo2 = insertion.cheapest_insertion(distance_matrix)
qc.evaluate_quality("Cheapest Insertion", algo2, distance_matrix, lower_bound)

# Choix du chemin de Nearest Neighbor pour le calcul des deux prochaines heuristiques
algo3 = twoopt.two_opt(algo1.copy(), distance_matrix)
qc.evaluate_quality("2-Opt", algo3, distance_matrix, lower_bound)

# Paramètres de Simulated Annealing
initial_temperature = 10000
cooling_rate = 0.995
path = algo1.copy()

path.pop() # On enlève le dernier 0 de Nearest Neighbor pour éviter les doublons

algo4_path, algo4_length = annealing.simulated_annealing(coords, path, initial_temperature, cooling_rate)
qc.evaluate_quality("Simulated Annealing", algo4_path, distance_matrix, lower_bound)

# Affichage sur la même figure des visualisations des 3 heuristiques
vs.plot_path(coords, [algo1, algo2, algo3], ["Nearest Neighbor", "Insertion", "Two Opt"])